In [39]:
import numpy as np
from medpy.io import load
import os

In [98]:
# Set path to test files and import
chosen_im = 'copd2'

# Set path to data folder containing copd1, copd2, etc. folders
data_dir = "../data"

#Set paths to landmarks and images
path_landmarks1 = os.path.join(data_dir, f"{chosen_im}/{chosen_im}_300_eBH_xyz_r1.txt")
path_landmarks2 = os.path.join(data_dir, f"{chosen_im}/{chosen_im}_300_iBH_xyz_r1.txt")
path_img1 = os.path.join(data_dir, f'./{chosen_im}/{chosen_im}_eBHCT.mha')
path_img2 = os.path.join(data_dir, f'./{chosen_im}/{chosen_im}_iBHCT.mha')

In [99]:
landmarks1 = np.loadtxt(path_landmarks1)
landmarks2 = np.loadtxt(path_landmarks2)
_, image_header1 = load(path_img1)

In [100]:
# Function to calculate TRE
def TRE(landmarks1, landmarks2, image_header1):
    """
    mean (and standard deviation) 3D Euclidean magnitude distance 
    between calculated and reference landmark positions for the 
    set of validation landmarks. All values are reported in units of millimeters
    
    :param landmarks1: (ndarry) first set of landmarks
    :param landmarks2: (ndarry) second set of landmarks
    :return: mean and SD 3D Euclidean magnitude distance
    """
    spacing = tuple(list(image_header1.GetSpacing()))
    landmarks1 = spacing*landmarks1
    landmarks2 = spacing*landmarks2
    diff = landmarks1 - landmarks2
    squared = diff * diff
    dist = np.sqrt(np.sum(squared,axis=1))
    mean_TRE = np.mean(dist)
    sd_TRE = np.std(dist)

    return dist, mean_TRE, sd_TRE

In [101]:
dist, mean_TRE, SD_TRE = TRE(landmarks1, landmarks2, image_header1)
print(mean_TRE, SD_TRE)

21.64084010925038 6.415267676867441
